## 0. Setup

In [36]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from collections import OrderedDict

In [16]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [17]:
# Set up random seed to 1008. Do not change the random seed.
seed = 1008
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed_all(seed)

In [18]:
def NLLLoss(logs, targets):
    out = torch.zeros_like(targets, dtype=torch.float)
    print(logs)
    for i in range(len(targets)):
        out[i] = logs[i][targets[i]]
    print(out)
    return -out.sum()/len(out)

x = torch.randn(3, 5)
y = torch.LongTensor([0, 1, 2])
cross_entropy_loss = torch.nn.CrossEntropyLoss()
log_softmax = torch.nn.LogSoftmax(dim=1)
x_log = log_softmax(x)

nll_loss = torch.nn.NLLLoss()
print("Torch CrossEntropyLoss: ", cross_entropy_loss(x, y))
print("Torch NLL loss: ", nll_loss(x_log, y))
print("Custom NLL loss: ", NLLLoss(x_log, y))

Torch CrossEntropyLoss:  tensor(1.8024)
Torch NLL loss:  tensor(1.8024)
tensor([[-2.3975, -1.8599, -0.8858, -1.5456, -2.0572],
        [-4.1107, -1.3640, -1.1521, -2.0161, -1.2772],
        [-1.2443, -1.4025, -1.6459, -2.6446, -1.5994]])
tensor([-2.3975, -1.3640, -1.6459])
Custom NLL loss:  tensor(1.8024)


In [19]:
# Load the MNIST training set with batch size 128, apply data shuffling and normalization
batch_size=128
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [20]:
# Load the MNIST test set with batch size 128, apply data shuffling and normalization
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [21]:
def train(model, device, train_loader, optimizer, epoch, log_interval = 100):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):
    
        # Send data and target to device
        data, target = data.to(device), target.to(device)

        # Zero out the optimizer
        optimizer.zero_grad()
        
        # Pass data through model
        output = model(data)

        # Compute the negative log likelihood loss
        loss = F.nll_loss(output, target)
        
        # Backpropagate loss
        loss.backward(retain_graph=True)
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [22]:
# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    with torch.no_grad():
        
        # Loop through data points
        for data, target in test_loader:
        
            # Send data to device
            data, target = data.to(device), target.to(device)
        
            # Pass data through model
            output = model(data)
            
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
            
            # Get predictions from the model for each data point
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
            
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    
    # Compute the average test_loss
    avg_test_loss = test_loss / len(test_loader.dataset)
    
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

In [37]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5, 5))
        self.conv3 = nn.Conv2d(16, 120, kernel_size=(5, 5))
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        print(f'1.{x.size()}')
        x = F.relu(self.conv1(x))
        print(f'2.{x.size()}')
        x = F.max_pool2d(x, kernel_size=(2, 2), stride=2)
        print(f'3.{x.size()}')
        x = F.relu(self.conv2(x))
        print(f'4.{x.size()}')
        x = F.max_pool2d(x, kernel_size=(2, 2), stride=2)
        x = F.relu(self.conv3(x))
        print(f'5.{x.size()}')
        x = x.view(-1, 120) # reshaping
        x = F.relu(self.fc1(x))
        print(f'6.{x.size()}')
        x = self.fc2(x)
        print(f'7.{x.size()}')
        x = F.log_softmax(x, dim=1)
        return x

In [40]:
image = torch.randn(3, 10, 20)
print(image.size())
print(image.view(-1).size())

torch.Size([3, 10, 20])
torch.Size([600])
